In [ ]:
# import necessary libraries

import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import glob

from skimage.filters.rank import entropy
from scipy import stats
from skimage.morphology import disk
from skimage import io
from skimage.filters import threshold_otsu

In [ ]:
def img_reading(path):
    """
    Reads images in several folders in a directory, thresholds them and applies entropy.
    
    Parameters
    ----------
    conc: string, concentration used for the images
    path: string, path to the directory where all the folder containing images are saved
    
    Returns
    ----------
    A dictionary whose key is the concentration and values are the measured gaps of the images
    """
    
    final = {}
    for folders in os.listdir(path):
        measures = []
        for imgs in glob.glob(path + folders + "/*.*"):
            img = io.imread(imgs, as_gray = True)
            entropy_img = entropy(img, disk(10))
            thresh = threshold_otsu(entropy_img)
            binary = entropy_img <= thresh
            measures.append(np.sum(binary == 1))
            
        final[folders] = measures
        
    return final

In [ ]:
def file_conversion(conc, measurements, file_name):
    """
    Saves the measrued data into a text and csv files.
    
    Parameters
    ----------
    conc: string, concentration of used on the images
    measureements: dict, provided by img_reading function
    file_name: string, name of the saved file
    
    Returns
    ----------
    Text and csv files in the directory
    """
    
    file = open(file_name + ".txt", "a")
    file.write(conc)
    file.write(":\n")
    for k1, v1 in measurements.items():
        file.write(k1)
        file.write(":")
        file.write(str(v1))
        file.write("\n")
        
    file.write("\n\n")
    file.close()
    
    df = pd.DataFrame({conc: measurements})
    df.to_csv(file_name + ".csv", mode = "a")

In [ ]:
def df_reading(csv_file):
    """
    Reads the csv file saved and returns a dictionary whose keys are the concentrations of the experiements and
    values are the measured gap areas of each image.
    
    Parameters
    ----------
    csv_file: csv, csv file created by file_conversion function
    
    Returns
    ----------
    A dictionary
    """
    
    df = pd.read_csv(file)
    df = df.rename(columns = {"Unnamed: 0": "Wells"})
    df = df.set_index("Wells")
    measures = {}
    
    for col in df.columns:
        for k, v in df[col].to_dict().items():
            if str(v) == "nan":
                continue

            elif col not in measures:
                cv = list(map(int, v.strip("][").split(", ")))
                measures[col] = np.array(cv)

            else:
                cv = list(map(int, v.strip("][").split(", ")))
                measures[col] = np.vstack((measures[col] , np.array(cv)))
                
    return measures

In [ ]:
def slope_measurements(measurements_dicts):
    """
    Measures the slope of a line between two dtaa points
    
    Parameters
    ----------
    measurements_dicts: dictionary, provided by df_reading function
    
    Returns
    ----------
    A dictionary whose keys are the same as measurements dicts and values are the slopes of the lines
    """
    end = 2
    slope_dict = {}

    for k, v in shortened_dict.items():
        for start in range(0, len(v) - 1):
            if k not in slope_dict:
                slope_dict[k] = [0, stats.linregress([0, 1], v[start: end])[0]]
                end += 1
                
            else:
                slope_dict[k].append(stats.linregress([0, 1], v[start: end])[0])
                end += 1
        end = 2
        
    return slope_dict

In [ ]:
def slope_visualazation(slopes, time, x_label, y_label, title, file_name):
    """
    Graphs the slopes of the lines
    
    Parameters
    ----------
    slopes: dictionary, provided by slope_measurements functions
    time: list, use list(range(starting_time, ending_time + 1)) to create the time that will be used on x axis
    x_label: string, label for x axis
    y_label: string, label for y axis
    title: string, title of the figure
    file_name: string, used to save the file with
    
    Returns
    -------
    A png file in the directory
    """
    plt.figure(figsize = (10, 7))
    for k, v in slopes.items():
        plt.plot(time, v, label = k)
    
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.legend(loc = "upper right", bbox_to_anchor = (1.15, 1.01))
    plt.title(title)
    plt.savefig(file_name)